In [1]:
import findspark
findspark.init('/home/yerke/spark-3.3.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [9]:
data = spark.read.csv('../course_data/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [10]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [12]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [15]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [16]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [18]:
assembler = VectorAssembler(inputCols= ['Avg Session Length',
                                        'Time on App',
                                        'Time on Website',
                                        'Length of Membership'],
                           outputCol='features')

In [19]:
output = assembler.transform(data)

In [20]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [22]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [23]:
final_data = output.select('features', 'Yearly Amount Spent')

In [24]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [25]:
train_data, test_data = final_data.randomSplit([.7, .3])

In [26]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                352|
|   mean|  500.1575244632891|
| stddev|   80.0804997607848|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [27]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                148|
|   mean| 497.30790890687626|
| stddev|  77.69455453432253|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [40]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [41]:
lr_model = lr.fit(train_data)

In [42]:
test_results = lr_model.evaluate(test_data)

In [43]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -4.928341634485207|
| 10.676361784064625|
| -3.683130623700663|
|  -8.99202115110569|
|  3.836673050026377|
|  18.19093200747784|
| 17.380295457350314|
|  6.764132313201742|
| -3.322298228272757|
| -17.39575816455755|
| -9.163717772009875|
| 1.1769922618818782|
|-1.6997028848307991|
| -10.02421661510823|
|-1.5827377273628258|
| 12.032156588160262|
|-13.667410125385686|
|  5.194342928620358|
|-2.0391061381810687|
| 0.4101142580765327|
+-------------------+
only showing top 20 rows



In [44]:
test_results.rootMeanSquaredError

10.656876846278188

In [45]:
test_results.r2

0.9810581186176385

In [46]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [47]:
unlabeled_data = test_data.select('features')

In [48]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.7377203726281...|
|[31.2681042107507...|
|[31.5261978982398...|
|[31.5316044825729...|
|[31.6005122003032...|
|[31.6098395733896...|
|[31.6548096756927...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8279790554652...|
|[31.8293464559211...|
|[31.8627411090001...|
|[31.8648325480987...|
|[31.9120759292006...|
|[31.9262720263601...|
|[31.9365486184489...|
|[32.0305497162129...|
|[32.0775900443291...|
|[32.0961089938451...|
+--------------------+
only showing top 20 rows



In [49]:
predictions = lr_model.transform(unlabeled_data)

In [50]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 287.3995873543997|
|[30.7377203726281...|451.10438041216526|
|[31.2681042107507...| 427.1536637975246|
|[31.5261978982398...| 418.0865473434435|
|[31.5316044825729...| 432.6789326793362|
|[31.6005122003032...| 460.9819194836191|
|[31.6098395733896...|427.16525419375785|
|[31.6548096756927...|468.49929141434677|
|[31.8124825597242...|   396.13264321207|
|[31.8164283341993...| 518.5182496682139|
|[31.8279790554652...| 449.1664653189514|
|[31.8293464559211...| 383.9753457260931|
|[31.8627411090001...| 557.9978440588775|
|[31.8648325480987...| 449.9154970919219|
|[31.9120759292006...|389.11745403307054|
|[31.9262720263601...|380.17277685616614|
|[31.9365486184489...| 440.8667950207139|
|[32.0305497162129...| 589.0801404899914|
|[32.0775900443291...| 403.0722413572871|
|[32.0961089938451...|374.98834115216664|
+--------------------+------------